In [1]:
import openpyxl
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

from matplotlib import pyplot as plt
import datetime
import math
import pandas as pd
from scipy import stats
import seaborn as sn

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [5]:
egg_1 = pd.read_pickle('RAW_DATA_HOURLY/Egg#1/2020-05-16 Download_egg0080446e6e980130_sampled_hourly.pkl')
egg_2 = pd.read_pickle('RAW_DATA_HOURLY/Egg#2/2020-05-16 Download_egg0080446fa82b0130_sampled_hourly.pkl')
egg_4 = pd.read_pickle('RAW_DATA_HOURLY/Egg#4/2020-05-16 Download_egg00804426a6980132_sampled_hourly.pkl')

In [6]:
sensor_cluster = pd.concat((egg_1, egg_2, egg_4))
sensor_cluster = sensor_cluster.groupby(sensor_cluster.index).mean()
sensor_cluster

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa]
Timestamp,,,,,,,,,,
2020-05-06 13:00:00,19.990000,22.908889,1.555556,2.222222,2.333333,142.283333,1.422313,2573.472222,1.405332,99264.800000
2020-05-06 14:00:00,18.079124,24.662431,5.363942,6.640385,7.081677,58.415422,1.423892,354.630470,1.420346,99265.046635
2020-05-06 15:00:00,22.104889,19.616056,1.189444,1.930556,2.090000,52.154389,1.424351,158.829056,1.421864,99188.698889
2020-05-06 16:00:00,21.898333,19.096778,1.161667,1.975000,2.187778,52.662000,1.424325,135.239611,1.422013,99140.480000
2020-05-06 17:00:00,21.836278,18.678667,1.469444,2.380556,2.604444,53.213833,1.424310,118.911944,1.422118,99111.335556
...,...,...,...,...,...,...,...,...,...,...
2020-05-16 06:00:00,12.695833,65.470278,5.828889,8.248889,8.559444,47.098278,1.423714,87.873556,1.421911,100012.624444
2020-05-16 07:00:00,13.162722,64.219667,4.227222,6.166111,6.403889,45.938667,1.423770,91.679722,1.421901,100117.775556
2020-05-16 08:00:00,19.256722,48.158944,3.688889,5.160556,5.350556,41.632111,1.424333,96.895667,1.422140,100151.730556


In [14]:
sensor_cluster.to_pickle('egg_cluster_2020-05-16_sampled_hourly.pkl')

In [15]:
merged_dataset = sensor_cluster
airpointer_dataframe = pd.read_pickle('RAW_DATA_HOURLY/airpointer_07Jun2020_hourly.pkl')

In [16]:
merged_dataset['Airpointer_NO[ppb]'] = airpointer_dataframe['NO']
merged_dataset['Airpointer_NO2[ppb]'] = airpointer_dataframe['NO2']
merged_dataset['Airpointer_NOx[ppb]'] = airpointer_dataframe['NOx']
merged_dataset['Airpointer_O3[ppb]'] = airpointer_dataframe['O3']
merged_dataset['Airpointer_PM10_conc[ppb]'] = airpointer_dataframe['PM10_conc']
merged_dataset['Airpointer_PM10-2.5_conc[ppb]'] = airpointer_dataframe['PM10-2.5_conc']
merged_dataset['Airpointer_PM2.5_conc[ppb]'] = airpointer_dataframe['PM2.5_conc']
merged_dataset['Airpointer_Air_Temperature[degC]'] = airpointer_dataframe['Air_Temperature']
merged_dataset['Airpointer_Relative_Humidity[percent]'] = airpointer_dataframe['Relative_Humidity']

In [17]:
merged_dataset

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa],Airpointer_NO[ppb],Airpointer_NO2[ppb],Airpointer_NOx[ppb],Airpointer_O3[ppb],Airpointer_PM10_conc[ppb],Airpointer_PM10-2.5_conc[ppb],Airpointer_PM2.5_conc[ppb],Airpointer_Air_Temperature[degC],Airpointer_Relative_Humidity[percent]
Timestamp,,,,,,,,,,,,,,,,,,,
2020-05-06 13:00:00,19.990000,22.908889,1.555556,2.222222,2.333333,142.283333,1.422313,2573.472222,1.405332,99264.800000,1.405500,3.092333,4.498500,39.439667,14.807833,10.762833,4.045167,12.164500,29.069333
2020-05-06 14:00:00,18.079124,24.662431,5.363942,6.640385,7.081677,58.415422,1.423892,354.630470,1.420346,99265.046635,1.235167,2.711000,3.915167,40.489167,12.815500,8.869667,3.944333,12.281333,28.194667
2020-05-06 15:00:00,22.104889,19.616056,1.189444,1.930556,2.090000,52.154389,1.424351,158.829056,1.421864,99188.698889,1.483333,3.901500,5.595000,40.472333,15.562333,11.270000,4.290667,12.369500,27.624167
2020-05-06 16:00:00,21.898333,19.096778,1.161667,1.975000,2.187778,52.662000,1.424325,135.239611,1.422013,99140.480000,1.919000,5.137333,6.889833,40.594000,19.666000,15.035500,4.626500,12.851333,26.497000
2020-05-06 17:00:00,21.836278,18.678667,1.469444,2.380556,2.604444,53.213833,1.424310,118.911944,1.422118,99111.335556,2.494167,7.393667,9.502500,39.201833,21.422167,16.256500,5.165667,12.170000,27.646167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-16 06:00:00,12.695833,65.470278,5.828889,8.248889,8.559444,47.098278,1.423714,87.873556,1.421911,100012.624444,0.874333,4.042333,4.917500,24.351333,7.762833,2.886000,4.877333,9.659000,82.094833
2020-05-16 07:00:00,13.162722,64.219667,4.227222,6.166111,6.403889,45.938667,1.423770,91.679722,1.421901,100117.775556,1.345667,3.718167,5.063500,26.181500,7.431167,2.638333,4.793333,13.422833,67.867000
2020-05-16 08:00:00,19.256722,48.158944,3.688889,5.160556,5.350556,41.632111,1.424333,96.895667,1.422140,100151.730556,0.721500,2.777167,3.498667,30.927333,7.586500,3.308667,4.278167,15.648833,57.744500


In [18]:
merged_dataset.to_pickle('master_dataset_2020-05-16_sampled_hourly.pkl')

In [19]:
# Remove target values less than -1
merged_dataset = merged_dataset.drop(merged_dataset[merged_dataset['Airpointer_NO2[ppb]'] < -1].index)
# Drop the first two outliers
merged_dataset = merged_dataset.iloc[2:]

In [20]:
merged_dataset

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa],Airpointer_NO[ppb],Airpointer_NO2[ppb],Airpointer_NOx[ppb],Airpointer_O3[ppb],Airpointer_PM10_conc[ppb],Airpointer_PM10-2.5_conc[ppb],Airpointer_PM2.5_conc[ppb],Airpointer_Air_Temperature[degC],Airpointer_Relative_Humidity[percent]
Timestamp,,,,,,,,,,,,,,,,,,,
2020-05-06 15:00:00,22.104889,19.616056,1.189444,1.930556,2.090000,52.154389,1.424351,158.829056,1.421864,99188.698889,1.483333,3.901500,5.595000,40.472333,15.562333,11.270000,4.290667,12.369500,27.624167
2020-05-06 16:00:00,21.898333,19.096778,1.161667,1.975000,2.187778,52.662000,1.424325,135.239611,1.422013,99140.480000,1.919000,5.137333,6.889833,40.594000,19.666000,15.035500,4.626500,12.851333,26.497000
2020-05-06 17:00:00,21.836278,18.678667,1.469444,2.380556,2.604444,53.213833,1.424310,118.911944,1.422118,99111.335556,2.494167,7.393667,9.502500,39.201833,21.422167,16.256500,5.165667,12.170000,27.646167
2020-05-06 18:00:00,18.713167,20.979889,1.914444,2.970556,3.246667,53.447722,1.424056,98.456389,1.422115,99124.504444,1.459167,10.763833,12.104500,35.107333,16.479000,11.466000,5.013333,11.409000,28.912333
2020-05-06 19:00:00,15.544500,24.716389,2.207222,3.381667,3.666111,53.151778,1.423809,91.426611,1.422016,99156.430556,0.409500,3.808167,4.204333,38.284333,13.103500,9.678500,3.426167,10.577667,26.920333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-16 06:00:00,12.695833,65.470278,5.828889,8.248889,8.559444,47.098278,1.423714,87.873556,1.421911,100012.624444,0.874333,4.042333,4.917500,24.351333,7.762833,2.886000,4.877333,9.659000,82.094833
2020-05-16 07:00:00,13.162722,64.219667,4.227222,6.166111,6.403889,45.938667,1.423770,91.679722,1.421901,100117.775556,1.345667,3.718167,5.063500,26.181500,7.431167,2.638333,4.793333,13.422833,67.867000
2020-05-16 08:00:00,19.256722,48.158944,3.688889,5.160556,5.350556,41.632111,1.424333,96.895667,1.422140,100151.730556,0.721500,2.777167,3.498667,30.927333,7.586500,3.308667,4.278167,15.648833,57.744500


In [22]:
merged_dataset = pd.read_pickle('master_dataset_2020-05-16_sampled_hourly.pkl')
merged_dataset

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa],Airpointer_NO[ppb],Airpointer_NO2[ppb],Airpointer_NOx[ppb],Airpointer_O3[ppb],Airpointer_PM10_conc[ppb],Airpointer_PM10-2.5_conc[ppb],Airpointer_PM2.5_conc[ppb],Airpointer_Air_Temperature[degC],Airpointer_Relative_Humidity[percent]
Timestamp,,,,,,,,,,,,,,,,,,,
2020-05-06 13:00:00,19.990000,22.908889,1.555556,2.222222,2.333333,142.283333,1.422313,2573.472222,1.405332,99264.800000,1.405500,3.092333,4.498500,39.439667,14.807833,10.762833,4.045167,12.164500,29.069333
2020-05-06 14:00:00,18.079124,24.662431,5.363942,6.640385,7.081677,58.415422,1.423892,354.630470,1.420346,99265.046635,1.235167,2.711000,3.915167,40.489167,12.815500,8.869667,3.944333,12.281333,28.194667
2020-05-06 15:00:00,22.104889,19.616056,1.189444,1.930556,2.090000,52.154389,1.424351,158.829056,1.421864,99188.698889,1.483333,3.901500,5.595000,40.472333,15.562333,11.270000,4.290667,12.369500,27.624167
2020-05-06 16:00:00,21.898333,19.096778,1.161667,1.975000,2.187778,52.662000,1.424325,135.239611,1.422013,99140.480000,1.919000,5.137333,6.889833,40.594000,19.666000,15.035500,4.626500,12.851333,26.497000
2020-05-06 17:00:00,21.836278,18.678667,1.469444,2.380556,2.604444,53.213833,1.424310,118.911944,1.422118,99111.335556,2.494167,7.393667,9.502500,39.201833,21.422167,16.256500,5.165667,12.170000,27.646167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-16 06:00:00,12.695833,65.470278,5.828889,8.248889,8.559444,47.098278,1.423714,87.873556,1.421911,100012.624444,0.874333,4.042333,4.917500,24.351333,7.762833,2.886000,4.877333,9.659000,82.094833
2020-05-16 07:00:00,13.162722,64.219667,4.227222,6.166111,6.403889,45.938667,1.423770,91.679722,1.421901,100117.775556,1.345667,3.718167,5.063500,26.181500,7.431167,2.638333,4.793333,13.422833,67.867000
2020-05-16 08:00:00,19.256722,48.158944,3.688889,5.160556,5.350556,41.632111,1.424333,96.895667,1.422140,100151.730556,0.721500,2.777167,3.498667,30.927333,7.586500,3.308667,4.278167,15.648833,57.744500


In [23]:
merged_dataset = merged_dataset.drop(merged_dataset[merged_dataset['Airpointer_NO2[ppb]'] < -1].index)
merged_dataset

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa],Airpointer_NO[ppb],Airpointer_NO2[ppb],Airpointer_NOx[ppb],Airpointer_O3[ppb],Airpointer_PM10_conc[ppb],Airpointer_PM10-2.5_conc[ppb],Airpointer_PM2.5_conc[ppb],Airpointer_Air_Temperature[degC],Airpointer_Relative_Humidity[percent]
Timestamp,,,,,,,,,,,,,,,,,,,
2020-05-06 13:00:00,19.990000,22.908889,1.555556,2.222222,2.333333,142.283333,1.422313,2573.472222,1.405332,99264.800000,1.405500,3.092333,4.498500,39.439667,14.807833,10.762833,4.045167,12.164500,29.069333
2020-05-06 14:00:00,18.079124,24.662431,5.363942,6.640385,7.081677,58.415422,1.423892,354.630470,1.420346,99265.046635,1.235167,2.711000,3.915167,40.489167,12.815500,8.869667,3.944333,12.281333,28.194667
2020-05-06 15:00:00,22.104889,19.616056,1.189444,1.930556,2.090000,52.154389,1.424351,158.829056,1.421864,99188.698889,1.483333,3.901500,5.595000,40.472333,15.562333,11.270000,4.290667,12.369500,27.624167
2020-05-06 16:00:00,21.898333,19.096778,1.161667,1.975000,2.187778,52.662000,1.424325,135.239611,1.422013,99140.480000,1.919000,5.137333,6.889833,40.594000,19.666000,15.035500,4.626500,12.851333,26.497000
2020-05-06 17:00:00,21.836278,18.678667,1.469444,2.380556,2.604444,53.213833,1.424310,118.911944,1.422118,99111.335556,2.494167,7.393667,9.502500,39.201833,21.422167,16.256500,5.165667,12.170000,27.646167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-16 06:00:00,12.695833,65.470278,5.828889,8.248889,8.559444,47.098278,1.423714,87.873556,1.421911,100012.624444,0.874333,4.042333,4.917500,24.351333,7.762833,2.886000,4.877333,9.659000,82.094833
2020-05-16 07:00:00,13.162722,64.219667,4.227222,6.166111,6.403889,45.938667,1.423770,91.679722,1.421901,100117.775556,1.345667,3.718167,5.063500,26.181500,7.431167,2.638333,4.793333,13.422833,67.867000
2020-05-16 08:00:00,19.256722,48.158944,3.688889,5.160556,5.350556,41.632111,1.424333,96.895667,1.422140,100151.730556,0.721500,2.777167,3.498667,30.927333,7.586500,3.308667,4.278167,15.648833,57.744500


In [24]:
# Drop the first two outliers
merged_dataset = merged_dataset.iloc[2:]
merged_dataset

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa],Airpointer_NO[ppb],Airpointer_NO2[ppb],Airpointer_NOx[ppb],Airpointer_O3[ppb],Airpointer_PM10_conc[ppb],Airpointer_PM10-2.5_conc[ppb],Airpointer_PM2.5_conc[ppb],Airpointer_Air_Temperature[degC],Airpointer_Relative_Humidity[percent]
Timestamp,,,,,,,,,,,,,,,,,,,
2020-05-06 15:00:00,22.104889,19.616056,1.189444,1.930556,2.090000,52.154389,1.424351,158.829056,1.421864,99188.698889,1.483333,3.901500,5.595000,40.472333,15.562333,11.270000,4.290667,12.369500,27.624167
2020-05-06 16:00:00,21.898333,19.096778,1.161667,1.975000,2.187778,52.662000,1.424325,135.239611,1.422013,99140.480000,1.919000,5.137333,6.889833,40.594000,19.666000,15.035500,4.626500,12.851333,26.497000
2020-05-06 17:00:00,21.836278,18.678667,1.469444,2.380556,2.604444,53.213833,1.424310,118.911944,1.422118,99111.335556,2.494167,7.393667,9.502500,39.201833,21.422167,16.256500,5.165667,12.170000,27.646167
2020-05-06 18:00:00,18.713167,20.979889,1.914444,2.970556,3.246667,53.447722,1.424056,98.456389,1.422115,99124.504444,1.459167,10.763833,12.104500,35.107333,16.479000,11.466000,5.013333,11.409000,28.912333
2020-05-06 19:00:00,15.544500,24.716389,2.207222,3.381667,3.666111,53.151778,1.423809,91.426611,1.422016,99156.430556,0.409500,3.808167,4.204333,38.284333,13.103500,9.678500,3.426167,10.577667,26.920333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-16 06:00:00,12.695833,65.470278,5.828889,8.248889,8.559444,47.098278,1.423714,87.873556,1.421911,100012.624444,0.874333,4.042333,4.917500,24.351333,7.762833,2.886000,4.877333,9.659000,82.094833
2020-05-16 07:00:00,13.162722,64.219667,4.227222,6.166111,6.403889,45.938667,1.423770,91.679722,1.421901,100117.775556,1.345667,3.718167,5.063500,26.181500,7.431167,2.638333,4.793333,13.422833,67.867000
2020-05-16 08:00:00,19.256722,48.158944,3.688889,5.160556,5.350556,41.632111,1.424333,96.895667,1.422140,100151.730556,0.721500,2.777167,3.498667,30.927333,7.586500,3.308667,4.278167,15.648833,57.744500


In [26]:
merged_dataset.to_pickle('master_dataset_2020-05-16_sampled_hourly_fixed.pkl')